Basado sobre https://medium.com/mlearning-ai/transfer-learning-with-transformers-trainer-and-pipeline-for-nlp-8b1d2c1a8c3d

In [2]:
# !pip install datasets

In [4]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to C:/Users/darko/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
raw_datasets["train"][58]

{'text': 'I thought this was a very clunky, uninvolving version of a famous Australian story. Heath Ledger and Orlando Bloom were very good in their roles, and gave their characters some personality; but the whole thing felt forced and mechanical.<br /><br />The beginning could have been a lot more involving; perhaps starting with a shootout, and then flashing back for a recap of how they got there or that sort of thing. And I felt like every scene was routinely predictable and signposted, like a very bad tv soap.<br /><br />I was really looking forward to this movie, and hoping for something a lot better. The only thing I can say in its favour is that it beats the Mick Jagger version, but not by much.',
 'label': 0}

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10))

small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6
  Number of trainable parameters = 108311810


  0%|          | 0/6 [00:00<?, ?it/s]

ModuleNotFoundError: No module named 'torch.fx'

In [8]:
trainer.save_model()

In [13]:
from transformers import pipeline# load from previously saved model
pipe = pipeline("text-classification", model="test_trainer", tokenizer="bert-base-cased")
pipe("This restaurant is awesome")

file test_trainer/config.json not found


OSError: Can't load config for 'test_trainer'. Make sure that:

- 'test_trainer' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'test_trainer' is not a path to a local directory with something else, in that case)

- or 'test_trainer' is the correct path to a directory containing a config.json file

